**Мы будем работать с базой данных StackOverflow — сервиса вопросов и ответов о программировании используя язык запросов SQL**

1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(id)
FROM stackoverflow.posts 
WHERE post_type_id=1 AND (score > 300 OR favorites_count >= 100);

2. Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
SELECT ROUND(AVG(count), 0)
FROM 
    (SELECT CAST(DATE_TRUNC('day', creation_date) AS date) AS dt,
     COUNT(id)
      FROM stackoverflow.posts
      WHERE post_type_id = 1
      GROUP BY dt) as subq
WHERE dt >= '2008-11-01' AND dt <= '2008-11-18';

3. Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users u
LEFT JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE u.creation_date::date = b.creation_date::date;

4. Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON p.user_id = u.id
JOIN stackoverflow.votes v ON p.id = v.post_id
WHERE u.display_name = 'Joel Coehoorn' AND p.score >= 1;

5. Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
       RANK() OVER (ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id;

6. Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT DISTINCT v.user_id AS id,
       COUNT(v.id) OVER (PARTITION BY v.user_id) AS count
FROM stackoverflow.votes v
JOIN stackoverflow.vote_types vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
ORDER BY count DESC,
         id DESC
LIMIT 10;

7. Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.  
Отобразите несколько полей:  
- идентификатор пользователя;  
- число значков;  
- место в рейтинге — чем больше значков, тем выше рейтинг.  
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.  
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
WITH profile AS 
                (SELECT DISTINCT user_id, 
                        COUNT(id) AS count
                 FROM stackoverflow.badges
                 WHERE DATE_TRUNC('day', creation_date::date) >= '2008-11-15' 
                 AND DATE_TRUNC('day', creation_date::date) <= '2008-12-15'
                 GROUP BY 1
                 ORDER BY 2 DESC
                 LIMIT 10)
SELECT *,
       DENSE_RANK() OVER (ORDER BY count DESC)
FROM profile
ORDER BY count DESC,
         user_id;

8. Сколько в среднем очков получает пост каждого пользователя?  
Сформируйте таблицу из следующих полей:  
- заголовок поста;  
- идентификатор пользователя;  
- число очков поста;  
- среднее число очков пользователя за пост, округлённое до целого числа.  
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT title, 
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id), 0) AS avg_score
FROM stackoverflow.posts
WHERE score != 0 AND title <> '';

9. Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT p.title
FROM stackoverflow.posts AS p
LEFT JOIN stackoverflow.users AS u ON u.id = p.user_id
WHERE u.id IN (SELECT b.user_id
                 FROM stackoverflow.badges AS b
                 JOIN stackoverflow.users as u ON u.id = b.user_id
               GROUP BY 1
               HAVING COUNT (DISTINCT b.id)>1000)
AND p.title IS NOT NULL;

10. Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:  
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;  
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;  
- пользователям с числом просмотров меньше 100 — группу 3.  
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
SELECT id,
       views,
       CASE 
           WHEN views >= 350 THEN 1
           WHEN views < 350 AND views >= 100 THEN 2
           WHEN views < 100 THEN 3
        END AS group
FROM stackoverflow.users
WHERE location LIKE '%United States%' AND views != 0;

11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH g AS (SELECT id,
                  views,
                  CASE 
                               WHEN views >= 350 THEN 1
                               WHEN views < 350 AND views >= 100 THEN 2
                               WHEN views < 100 THEN 3
                         END AS group
                         FROM stackoverflow.users
                         WHERE location LIKE '%United States%' AND views != 0),
mv AS (SELECT *,
              MAX(views) OVER (PARTITION BY g.group) AS max_v
       FROM g
       ORDER BY g.views DESC,
                id)
SELECT mv.id,
       mv.group,
       mv.views
FROM mv
WHERE views = max_v;

12. Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:  
- номер дня;  
- число пользователей, зарегистрированных в этот день;  
- сумму пользователей с накоплением.

In [ ]:
WITH fd AS 
          (SELECT *
           FROM stackoverflow.users
           WHERE DATE_TRUNC('day', creation_date)::date BETWEEN '2008-11-01' AND '2008-11-30'),
pf AS (SELECT DENSE_RANK () OVER (ORDER BY fd.creation_date::date) AS rank,
               COUNT(fd.id) AS cnt,
               COUNT(fd.id) OVER (ORDER BY DATE_TRUNC('day',fd.creation_date)::date) AS cnts
       FROM fd
       GROUP BY fd.creation_date, fd.id)
SELECT pf.rank,
       SUM(pf.cnt),
       pf.cnts
FROM pf
GROUP BY 1,
         3
ORDER BY 1;

13. Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:  
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
SELECT DISTINCT u.id,
       FIRST_VALUE(p.creation_date) OVER (PARTITION BY u.id ORDER BY p.creation_date) - u.creation_date
FROM stackoverflow.users u 
JOIN stackoverflow.posts p ON u.id = p.user_id;

14. Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT DATE_TRUNC('month', creation_date)::date,
       SUM(views_count)
FROM stackoverflow.posts
GROUP BY DATE_TRUNC('month', creation_date)::date
ORDER BY SUM(views_count) DESC;

15.  Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
    COUNT(DISTINCT user_id) as answ_count
FROM stackoverflow.posts p
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
JOIN stackoverflow.users u ON p.user_id = u.id
WHERE type = 'Answer' AND DATE_TRUNC('day', p.creation_date) >= DATE_TRUNC('day', u.creation_date) AND
    DATE_TRUNC('day', p.creation_date) <= (DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month')
GROUP BY 1
HAVING COUNT(*) > 100
ORDER BY 1;

16. Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
WITH users AS 
            (SELECT DISTINCT u.id
            FROM  stackoverflow.users u
            JOIN  stackoverflow.posts p ON u.id = p.user_id
            WHERE DATE_TRUNC('day', u.creation_date)::date BETWEEN '2008-09-01' AND '2008-09-30' 
            AND DATE_TRUNC('day', p.creation_date)::date BETWEEN '2008-12-01' AND '2008-12-31')
SELECT DATE_TRUNC('month', pp.creation_date)::date,
        COUNT(pp.id)
FROM users
JOIN stackoverflow.posts pp on pp.user_id=users.id
GROUP BY 1
ORDER BY 1 DESC;

17. Используя данные о постах, выведите несколько полей:  
- идентификатор пользователя, который написал пост;  
- дата создания поста;  
- количество просмотров у текущего поста;  
- сумму просмотров постов автора с накоплением.  
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY 1, 
         2;

18.  Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
WITH day AS 
           (SELECT DISTINCT user_id,
                   COUNT(DISTINCT creation_date::date) AS count_day
            FROM stackoverflow.posts
            WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
            GROUP BY 1)
SELECT ROUND(AVG(count_day),0)
FROM day;

19. На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:  
- номер месяца;  
- количество постов за месяц;  
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.  
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.


In [ ]:
WITH cp AS 
          (SELECT EXTRACT('month' FROM creation_date::date) AS dt,
                  COUNT(id) AS cnt_p
           FROM stackoverflow.posts
           WHERE DATE_TRUNC('month', creation_date)::date BETWEEN '2008-09-01' AND '2008-12-31'
           GROUP BY 1)
SELECT *,
      ROUND((cnt_p::numeric/(LAG(cnt_p) OVER (ORDER BY dt)) - 1) * 100, 2) AS posts_growth
FROM cp;

20. Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:  
- номер недели;  
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
SELECT EXTRACT(week FROM creation_date::date) AS week,
       MAX(creation_date) AS last_post
FROM stackoverflow.posts
WHERE user_id IN
                (SELECT user_id
                 FROM
                     (SELECT user_id,
                             COUNT(*)
                      FROM stackoverflow.posts
                      GROUP BY 1
                      ORDER BY 2 DESC
                      LIMIT 1) max_user )  
AND creation_date BETWEEN '2008-10-01' AND '2008-11-01'
GROUP BY EXTRACT(week FROM creation_date::date);